In [1]:
import xlrd
import math as m

#Oggetto nodo con attributi utili per lo svolgimento dell'algoritmo
class nodo():
	def __init__(self,ide,nome,lat,lon,stato,identificativo,defi,pot,coll,adia): 
		self.ide=ide
		self.lat=lat
		self.lon=lon
		self.nome=nome
		self.stato=stato
		self.identificativo=identificativo
		self.defi=defi
		self.pot=pot
		self.coll=coll
		self.adia=adia

#Funzione che calcola la distanza tra due punti su un piano sferico fornendole le coordinate di ciascun punto
def dist2punti(latA,lonA,latB,lonB):
	Rt=6378.388
	lat_A = (m.pi*latA)/180.
	lat_B = (m.pi*latB)/180.
	lon_A = (m.pi*lonA)/180.
	lon_B = (m.pi*lonB)/180.
	dl=abs(lon_A-lon_B)

	p = m.acos((m.sin(lat_B)*m.sin(lat_A))+(m.cos(lat_B)*m.cos(lat_A)*m.cos(dl)))
	if(lon_A>0 and lon_B>0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A<0 and lon_B<0): 
		if(lon_B > lon_A): 
			p=p
		else:
			p=float('inf')
	if(lon_A>0 and lon_B<0):
		if((lon_A+abs(lon_B))<m.pi): 
			p=float('inf')
		else:
			p=p
	if(lon_A<0 and lon_B>0):
		if((lon_B+abs(lon_A))<m.pi): 
			p=p
		else:
			p=float('inf')
	return(p*Rt) 

#Lettura database
dati=xlrd.open_workbook('City.xlsx')
foglio=dati.sheet_by_name('Foglio1')
nR=foglio.nrows


#Imposto la città iniziale
citta_inizio="New York"
inizio="1840034016"


#Creo una serie di liste che conterranno i dati utili per losvolgimento del progetto
nome=[]
lat=[]
log=[]
stato=[]
stato2=[]
pop=[]
iden=[]
pesi=[]
nodi=[]

#Riempio le liste con i dati contenuti nel database
for i in range(1,nR):
	nome.append(foglio.cell(i,0).value)
	lat.append(foglio.cell(i,2).value)
	log.append(foglio.cell(i,3).value)
	stato.append(foglio.cell(i,6).value)
	stato2.append(foglio.cell(i,7).value)
	pop.append(foglio.cell(i,9).value)
	iden.append(str(int(foglio.cell(i,10).value)))

print("Lettura dati da file terminata\n")

#Controllo che non ci siano città per le quali non sia stata definita la numerosità della popolazione, altrimenti metto a zero
for i in range(0,nR-1):
	if(pop[i]==''):
		pop[i]=0

print("Controllo popolazioni terminato\n")

#Elimino tutti i record con valori errati (non ci possono essere due città con stesse coordinate)
elim=[]
print("Città con errori\n")
for i in range(0,nR-1):
	for j in range(0,nR-1):
		if(lat[j]==lat[i] and log[j]==log[i] and iden[j]!=iden[i]):
			print(nome[j],iden[j])
			elim.append(j)
print("\n")
            
for j in elim:
	print("rimosso per coordinate coincidenti: ",iden[j])    
	nome.pop(j)
	lat.pop(j)
	log.pop(j)
	stato.pop(j)
	stato2.pop(j)
	pop.pop(j)
	iden.pop(j)
print("\n")
#Aggiorno la numerosità dopo aver pulito i dati
nR=len(nome)

#Imposto la cittàfinale come copia di quella iniziale
citta_fine=citta_inizio+"_fine"
fine="1111111111"

for i in range(0,nR):
	if(iden[i]==inizio):
		nome.append(citta_fine)
		lat.append(lat[i])
		log.append(log[i]-0.1)
		stato.append(stato[i])
		stato2.append(stato2[i])
		pop.append(pop[i])
		iden.append(fine)

#Aggiorno la numerosità con l'aggiunta di quella finale
print("Numerosità città:\n")
print(len(nome))
print("\n")
nN=len(nome)

#ciclo per calcolare le distanze e quindi i pesi di ogni città (con il calcolo di questi dati l'oggetto nodo è completo)
print("Assegnazione pesi in corso...\n")
for i in range(0,nN):
	vicini=[]
	for j in range(0,nN):
		if(iden[i]!=iden[j]):
			d=dist2punti(lat[i],log[i],lat[j],log[j])
			if(len(vicini)==0):
				vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
			else:	
				for k in range(0,len(vicini)):
					if(d<vicini[k][1]):
						vicini.insert(k,[iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])
						if(len(vicini)>3):					
							vicini.pop(3)
						break
					if(len(vicini)<3):
						vicini.append([iden[j],d,pop[j],stato[i]+"-"+stato2[i],stato[j]+"-"+stato2[j]])

	diz={}
	for l in range(0,len(vicini)):
		pes=2**(l+1)
		if(vicini[l][2]>200000):
			pes=pes+2
		if(vicini[l][3] != vicini[l][4]):
			pes=pes+2
		diz[vicini[l][0]]=pes 

	print("Ancora "+str(nN-i)+" citta' rimanenti" )

	pesi.append(diz)

	nodi.append(nodo(i,nome[i],lat[i],log[i],stato[i]+"-"+stato2[i],iden[i],True,float('inf'),"",pesi[i])) 
print("\n")

#Algoritmo per il calcolo del percorso minimo in un grafo
print("Calcolo del grafo utilizzando l'algoritmo Dijkstra\n")
print("Città di partenza: " + citta_inizio)
print("\n")
print("Elenco città controllate\n")

#Imposto il primo nodo del grafo
for x in nodi:
	if(x.identificativo == inizio):
		x.defi=False
		x.pot=0
		x.coll=inizio
		i=x.ide

#Partendo dal primo controllo tutti i nodi plausibili per far parte delpercorso minimo
while True:
	for x in nodi[i].adia.keys():
		for y in nodi:
			if (y.identificativo == x and y.defi ):
				somma=nodi[i].pot + nodi[i].adia[x]
				if(y.pot>somma):
					y.pot=somma
					y.coll=nodi[i].identificativo
	m=float('inf')
	for z in nodi:
		if(z.defi and z.pot<m):
			m=z.pot
			n=z.ide
	nodi[n].defi=False
	i=n
	print(nodi[i].nome)
	if(nodi[i].identificativo == fine ):
		lung=nodi[i].pot
		break

#Due liste: una per il percorsocon gli id e l'altra con i nomi delle città        
percorso=[]
percorso_nomi=[]


#Trovoil percorso minimo a ritroso
s=fine
while True:
	percorso.append(s)
	for x in nodi:
		if(x.identificativo == s):
			s=x.coll
	if(s==inizio):
		percorso.append(s)
		break

#Converto gli id in nomi per l'output
for x in percorso:
	for y in nodi:
		if(y.identificativo==x):
			percorso_nomi.append(y.nome+" ("+y.stato+")")

#Utilizzo una stringa per contenere le tappe del percorso inserite dalla partenza all'arrivo            
strada=""
for i in range(0,len(percorso_nomi)-1):
	strada=strada+percorso_nomi[len(percorso_nomi)-1-i]+" - "
strada=strada+" "+percorso_nomi[0]

#Stampo il risultato con tutte le tappe del percorso minimo, i giorni e le ore impiegate per completarlo
print("-------------------Risultato----------------------------")
print("Percorso terminato in "+str(lung)+" ore o "+str(float(lung)/24.)+" giorni. (tempo minimo)" )
print("Tragitto: " + strada)
print("--------------------------------------------------------")

Lettura dati da file terminata

Controllo popolazioni terminato

Città con errori

San Pedro Garza García 1484228590
Chicoloapan 1484980825
Kallithéa 1300675902
Santa Catarina Ayotzingo 1484300687
Chimalhuacán 1484504936
Altamira 1484764524
Monterrey 1484559591
Piraeus 1300458367
Don Bosco 1032701626
Guasavito 1484662260
Miramar 1484353090
José Mármol 1032126414
Kojskë 1901791413
Vranjak 1901861427
Rafael Calzada 1032898708
Qiryat Yam 1376460777
Bernal 1032030395
Altofts 1826413238
Guasave 1484450749
Lipjan 1901682048
Rahovec 1901336358
Qiryat Bialik 1376833983
South Park Township 1840073713
South Park 1840150312
Chalco 1484834149
Normanton 1826795957


rimosso per coordinate coincidenti:  1484228590
rimosso per coordinate coincidenti:  1484980825
rimosso per coordinate coincidenti:  1418863956
rimosso per coordinate coincidenti:  1840142185
rimosso per coordinate coincidenti:  1484504936
rimosso per coordinate coincidenti:  1840000395
rimosso per coordinate coincidenti:  1484559591
ri

Ancora 26333 citta' rimanenti
Ancora 26332 citta' rimanenti
Ancora 26331 citta' rimanenti
Ancora 26330 citta' rimanenti
Ancora 26329 citta' rimanenti
Ancora 26328 citta' rimanenti
Ancora 26327 citta' rimanenti
Ancora 26326 citta' rimanenti
Ancora 26325 citta' rimanenti
Ancora 26324 citta' rimanenti
Ancora 26323 citta' rimanenti
Ancora 26322 citta' rimanenti
Ancora 26321 citta' rimanenti
Ancora 26320 citta' rimanenti
Ancora 26319 citta' rimanenti
Ancora 26318 citta' rimanenti
Ancora 26317 citta' rimanenti
Ancora 26316 citta' rimanenti
Ancora 26315 citta' rimanenti
Ancora 26314 citta' rimanenti
Ancora 26313 citta' rimanenti
Ancora 26312 citta' rimanenti
Ancora 26311 citta' rimanenti
Ancora 26310 citta' rimanenti
Ancora 26309 citta' rimanenti
Ancora 26308 citta' rimanenti
Ancora 26307 citta' rimanenti
Ancora 26306 citta' rimanenti
Ancora 26305 citta' rimanenti
Ancora 26304 citta' rimanenti
Ancora 26303 citta' rimanenti
Ancora 26302 citta' rimanenti
Ancora 26301 citta' rimanenti
Ancora 263

Ancora 26059 citta' rimanenti
Ancora 26058 citta' rimanenti
Ancora 26057 citta' rimanenti
Ancora 26056 citta' rimanenti
Ancora 26055 citta' rimanenti
Ancora 26054 citta' rimanenti
Ancora 26053 citta' rimanenti
Ancora 26052 citta' rimanenti
Ancora 26051 citta' rimanenti
Ancora 26050 citta' rimanenti
Ancora 26049 citta' rimanenti
Ancora 26048 citta' rimanenti
Ancora 26047 citta' rimanenti
Ancora 26046 citta' rimanenti
Ancora 26045 citta' rimanenti
Ancora 26044 citta' rimanenti
Ancora 26043 citta' rimanenti
Ancora 26042 citta' rimanenti
Ancora 26041 citta' rimanenti
Ancora 26040 citta' rimanenti
Ancora 26039 citta' rimanenti
Ancora 26038 citta' rimanenti
Ancora 26037 citta' rimanenti
Ancora 26036 citta' rimanenti
Ancora 26035 citta' rimanenti
Ancora 26034 citta' rimanenti
Ancora 26033 citta' rimanenti
Ancora 26032 citta' rimanenti
Ancora 26031 citta' rimanenti
Ancora 26030 citta' rimanenti
Ancora 26029 citta' rimanenti
Ancora 26028 citta' rimanenti
Ancora 26027 citta' rimanenti
Ancora 260

Ancora 25784 citta' rimanenti
Ancora 25783 citta' rimanenti
Ancora 25782 citta' rimanenti
Ancora 25781 citta' rimanenti
Ancora 25780 citta' rimanenti
Ancora 25779 citta' rimanenti
Ancora 25778 citta' rimanenti
Ancora 25777 citta' rimanenti
Ancora 25776 citta' rimanenti
Ancora 25775 citta' rimanenti
Ancora 25774 citta' rimanenti
Ancora 25773 citta' rimanenti
Ancora 25772 citta' rimanenti
Ancora 25771 citta' rimanenti
Ancora 25770 citta' rimanenti
Ancora 25769 citta' rimanenti
Ancora 25768 citta' rimanenti
Ancora 25767 citta' rimanenti
Ancora 25766 citta' rimanenti
Ancora 25765 citta' rimanenti
Ancora 25764 citta' rimanenti
Ancora 25763 citta' rimanenti
Ancora 25762 citta' rimanenti
Ancora 25761 citta' rimanenti
Ancora 25760 citta' rimanenti
Ancora 25759 citta' rimanenti
Ancora 25758 citta' rimanenti
Ancora 25757 citta' rimanenti
Ancora 25756 citta' rimanenti
Ancora 25755 citta' rimanenti
Ancora 25754 citta' rimanenti
Ancora 25753 citta' rimanenti
Ancora 25752 citta' rimanenti
Ancora 257

Ancora 25509 citta' rimanenti
Ancora 25508 citta' rimanenti
Ancora 25507 citta' rimanenti
Ancora 25506 citta' rimanenti
Ancora 25505 citta' rimanenti
Ancora 25504 citta' rimanenti
Ancora 25503 citta' rimanenti
Ancora 25502 citta' rimanenti
Ancora 25501 citta' rimanenti
Ancora 25500 citta' rimanenti
Ancora 25499 citta' rimanenti
Ancora 25498 citta' rimanenti
Ancora 25497 citta' rimanenti
Ancora 25496 citta' rimanenti
Ancora 25495 citta' rimanenti
Ancora 25494 citta' rimanenti
Ancora 25493 citta' rimanenti
Ancora 25492 citta' rimanenti
Ancora 25491 citta' rimanenti
Ancora 25490 citta' rimanenti
Ancora 25489 citta' rimanenti
Ancora 25488 citta' rimanenti
Ancora 25487 citta' rimanenti
Ancora 25486 citta' rimanenti
Ancora 25485 citta' rimanenti
Ancora 25484 citta' rimanenti
Ancora 25483 citta' rimanenti
Ancora 25482 citta' rimanenti
Ancora 25481 citta' rimanenti
Ancora 25480 citta' rimanenti
Ancora 25479 citta' rimanenti
Ancora 25478 citta' rimanenti
Ancora 25477 citta' rimanenti
Ancora 254

...

Moon
Homewood
Farragut
Trussville
Oxford
Norwalk
Ashland
Oregon
Valley
Comăneşti
Târgu Neamţ
Rājmahal
Tullahoma
Salonta
Fort Thomas
Long Beach
Washington Court House
Buhuşi
Ochakiv
Warren
Gardendale
Port Clinton
Powell
Manchester
New Kensington
Pell City
Bayou Blue
Newport
Cristuru Secuiesc
Destrehan
Raceland
Norton
Fredonia
Scottsboro
Alexandria
Boaz
Lambertville
Lenoir City
Fultondale
Logan
Mrkonjić Grad
Withamsville
Center Township
Pinson
Nagaya
Sandown
Ohio
Pea Ridge
Satsuma
Punxsutawney
Onagawa
Attalla
Tellico Village
Sweetwater
Vienna
Vernon
Mount Carmel
Fălciu
Fox Chapel
Shaoxing
Xushan
Yueqing
Raipur
Shanhu
Morioka
Oradea
Rāiganj
Yangmei
Bacău
Kamagaya
Vršac
West View
Bellevue
Smederevska Palanka
Lučani
Kovin
Anniston
Kashima
Sandusky
Upper Arlington
Vestavia Hills
Kamaishi
Shima
Rome
Wooster
Takahagi
Londonderry
Dunkirk
Greeneville
Allison Park
Helena
Rikuzen-Takata
Durleşti
Katsuura
Daigo
Circleville
McMinnville
Worthington
Moss Point
River Ridge
Irondale
Dubak
Winchester
Dăr

Huntingdon
Sinaia
Ianca
Stuarts Draft
Benner
Bumbeşti-Jiu
Gresham Park
Locust Grove
Vinton
Târgu Cărbuneşti
East Huntingdon
Amara
Perry
Roaring Spring
Elkin
Jasper
Bujor
Eastman
Copşa Mică
Harris
Lake Park
Novaci-Străini
Jackson
Walkertown
Sawmills
East Flat Rock
Kolkāta
Dhaka
Taipei
Jeonju
Asheville
Iksan
Craiova
Yilan
Hābra
Lynchburg
Gudivāda
Ginowan
Slobozia
Wangdue Phodrang
Žagubica
Zhemgang
Bāpatla
Weirton
Stonecrest
Hinesville
Repalle
Milton
Trebinje
Kernersville
Dublin
Baldwin
Bolhrad
Holly Springs
Cisnădie
Vinings
Bellefonte
Comarnic
Folkston
Panthersville
Iernut
Fishersville
Black Mountain
Swainsboro
Spring Township
Big Flats
Live Oak
Sărmaşu
Gulf Breeze
Bullskin
Însurăţei
Mayfield
Rocky Mount
Ocean City
Višegrad
Adams Township
Vishākhapatnam
Bārāsat
Hickory
Madhyamgram
Narsingdi
Brăila
Machilīpatnam
Nantou
Basīrhat
Bānsbāria
Hualien
Kalyani
Koja
Bor
Priboj
Leposaviq
Blace
Ćićevac
Brus
Trongsa
Plužine
Alpharetta
Elmira
Bhīmunipatnam
Statesboro
Martinsville
Milledgeville
Canton

Laurel
Stallings
Newberry
Dunn
Nishinoomote
Novi Pazar
Pucioasa
Parker
Holly Hill
Fruitland Park
Coal
Forest Acres
Progress
Triangle
Lower Swatara
Williamstown
General Toshevo
Selma
Marvin
Chamberlayne
Windsor
Nashville
Fort Lee
Newport
Conklin
Ettrick
Colesville
Durham
Leesburg
Itānagar
Istog
Bull Run
Virac
Dimitrovgrad
Gusinje
Jorhāt
Rocky Mount
Goldsboro
Indian Trail
Smithfield
Petersburg
Morrisville
Hanahan
Karlovo
Sevlievo
Clayton
Buftea
North Adams
Waxhaw
Neabsco
Glen Allen
Matiçan
Zimnicea
Balchik
Bolintin Vale
South Daytona
Colonial Park
Ōsako
James Island
Weddington
Smithfield
Nanticoke
Woodfield
Hartford
Butler Township
La Plata
Middletown
Gloucester Point
Vau i Dejës
Sangaree
Ashland
Higashikyōshin
Hamlet
Paxtonia
Sidney
Shepherdstown
Ranson
Charleston
Imphāl
Newport News
Sorsogon
Dibrugarh
Kohīma
Gabrovo
Kamenicë
Sudley
Plav
Vlasotince
Mount Pleasant
Waldorf
Port Orange
Hazleton
Wilson
Lower Paxton
Goose Creek
Monroe
New Smyrna Beach
Kinston
Greenfield
Haka
Laurinburg
Shena

Guankou
Springfield
Lianran
Deltona
Mati
Elbasan
Lipjan
Aspen Hill
Tysons
Merrifield
Glassmanor
Phôngsali
Leisure World
Probištip
Tearce
Centar Župa
Uzunköprü
Vranjak
Pikesville
Samokov
Pasadena
Lochearn
Merrimack
Palmer
Hudson
Panagyurishte
Shippensburg
Webster
East Lampeter
Auburn
Tsukumiura
Southchase
Azalea Park
Beacon
Oxford
Union Park
Cornwall
Ocean Pines
Pepperell
Travilah
Groton
Millsboro
Elhovo
East Cocalico
Bithlo
Dunn Loring
Cape St. Claire
Hamilton Township
Esopus
Indian Harbour Beach
Seaford
North Coventry
West Boylston
Hollis
Ogden
Robeson
Wawayanda
Belle Isle
Earl
Upper Mount Bethel
Straldzha
Harvard
Darnestown
Wescosville
Lower Nazareth
Willington
Hellertown
Penn Estates
Upper Pottsgrove
Wappingers Falls
Fairview
Bangor
Lake Township
Kunming
Meishan
Xichang
Jiannan
Poughkeepsie
Ternate
Pottstown
Pazardzhik
Gaithersburg
North Bethesda
Kičevo
Palm Tree
Idylwood
Glenmont
Hillcrest Heights
Delčevo
Wilson
Mamushë
Louang Namtha
Sofifi
Alafaya
Uwajima
Glen Burnie
Kingston
Wood

Byram
Middlebury
Upper Deerfield
Pleasantville
Green Brook
Plymouth Meeting
Edgemoor
Wharton
Finderne
Bursa
Zhongshu
Qingzhen
Trenton
Việt Trì
Chishui
Atlantic City
Körfez
Yalova
Sakon Nakhon
Strumica
Veles
Bắc Kạn
Loei
West Chester
Ban Houayxay
Ardmore
Vanimo
Bound Brook
Dunellen
Ilinden
Rosoman
Zelenikovo
Mogila
Clarkstown
Shelton
Newark
Mae Sot
Naugatuck
Springettsbury
Upper Moreland
West Goshen
Watertown
Wilton
Whitemarsh
Stuart
Ramsey
Willow Grove
Hopatcong
Hobe Sound
Berkeley Heights
Oxford
Oakland
North Castle
New Garden
Wanaque
East Greenwich
Weston
Sleepy Hollow
Kinnelon
Pittsgrove
Edgewater Park
Claymont
Brigantine
Pike Creek
Bilisht
Morris Plains
Logan
Oreland
Commercial
Chongqing
Gölcük
Kavadarci
Drexel Hill
Nakhon Phanom
Wewak
Phichit
North Plainfield
Morristown
Tak
Glenside
Kennett Square
Penn Wynne
Vĩnh Yên
Petrovec
Tari
Norwalk
Parsippany-Troy Hills
Ye
Galloway
Trumbull
Willingboro
Millville
Ridgefield
Burlington
Morris
Little Egg Harbor
Springfield Township
Franklin To